# Cython functions
* Cython supports 3 kinds of functions:
  * Python `def` functions -- compiled Python functions that work with Python types.
  * C-level `cdef` functions -- low-overhead C-level functions that support C-only types.
  * Hybrid `cpdef` functions -- C-level function with auto-generated Python compatibility wrappers.

In [1]:
%load_ext Cython

## `def` functions in Cython: identical to `builtin_function_or_method`

* Expects Python objects as inputs; always returns a Python object.

In [2]:
def pyfunc(a, b):
    """Adds its arguments polymorphically, in pure Python."""
    return a + b

print(type(pyfunc))
print(dir(pyfunc))

<class 'function'>
['__annotations__', '__call__', '__class__', '__closure__', '__code__', '__defaults__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__get__', '__getattribute__', '__globals__', '__gt__', '__hash__', '__init__', '__kwdefaults__', '__le__', '__lt__', '__module__', '__name__', '__ne__', '__new__', '__qualname__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__']


In [3]:
%%cython

def cyfunc(a, b):
    """Adds its arguments polymorphically, in Cython!"""
    return a + b

print(type(cyfunc))
print(dir(cyfunc))

LinkError: command 'C:\\Program Files (x86)\\Microsoft Visual Studio\\VC98\\bin\\link.exe' failed with exit status 1113

### Cython functions don't have dynamic attributes that Python functions have

In [ ]:
set(dir(pyfunc)) - set(dir(cyfunc))

In [ ]:
assert pyfunc(1, 2) == cyfunc(1, 2)
assert pyfunc('a', 'b') == cyfunc('a', 'b')

class klass:
    def __add__(self, other):
        print("Salut!")
        return 42

print(cyfunc(klass(), klass()))

## `cdef` functions: C-functions with Python-like syntax

In [ ]:
%%cython

cdef untyped(a, b):
    return a + b

print(untyped(1, 2), untyped('a', 'b'))

cdef int typed(double a, double b):
    return <int>(a + b) # Type casting in Cython.

print(typed(1, 2), typed(3.14, 2.72))

# print(typed('a', 'b')) <<<<<<<<<<< Compilation Error, not ValueError / RuntimeError.

### `cdef` variables / functions not visible to Python outside defining scope

In [ ]:
untyped(1, 2), typed(1, 2)

### Pop quiz:

* Conjecture what Cython does when executing `print(typed(1, 2))` to make the `print()` call work.

## `cpdef` functions: two functions in one!

In [ ]:
%%cython -a
    

# cpdef functions are just like cdef functions with
# an implicitly defined Python wrapper for free.
cpdef int cpdef_func(int y, int z):
    return y + z

# Call directly from other Cython code:
print(cpdef_func(1, 2))

In [ ]:
# Call from Python (via Python wrapper)
cpdef_func(1, 2)

### Pop quiz:

* given that a `cpdef` function is like a `cdef` function with the constraint of using only Python-convertible types in its signature, what C types can we not use with a `cpdef` function?

## `cdef` and `cpdef` functions and error propogation

In [ ]:
%%cython

cpdef int unchecked_div(int a, int b):
    return <int>(a / b)

# This result is concerning...
print(unchecked_div(1, 0))

In [ ]:
%%cython

cpdef int checked_div(int a, int b) except *:
    return <int>(a / b)

# This is better...
print(checked_div(1, 0)) # ZeroDivisionError: float division

### Raising exceptions inside cpdef / cdef functions with C return types

In [ ]:
%%cython

cpdef int func() except -1: # we guarantee that -1 is never a valid return value, 
    # ...                   # so Cython can use it as a sentinel to flag that an 
    raise ValueError("...") # exception has occurred.

In [ ]:
func()